# Data Preprocessing

Student namen: Puck, Sally, Jelle, Issa

Groep nummer: R2

De volgende code is gebruikt om de datasets voor te bereiden in bestanden die gebruikt zijn voor de datastory.

In [1]:
#importeren relevanten bibliotheken
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
#voorbeeld laten zien in werkgroep 

#Stap 1: Laad en filter de relevante datasets
data_raw = pd.read_csv('migranten_eu.csv')
data_ter = pd.read_csv('terrorism_eu.csv').rename(columns={'country_txt': 'Country of asylum', 'iyear': 'Year', 'multiple': 'Terrorist attacks'})
data_lon_lat = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv').rename(columns={'country': 'Country of asylum'})

df_ter = pd.read_csv('globalterrorismdb_0718dist.csv', encoding='latin-1')
df_ter.head()
df1_ter = df_ter.groupby(['iyear', 'region_txt', 'country_txt', 'gname']).count().reset_index()
df1_ter
mask = df1_ter[df1_ter['region_txt'].isin(['Eastern Europe', 'Western Europe'])]
mask
df2_ter = mask[mask.iyear.between(2000, 2017)]
df2_ter
df3_ter = df2_ter.groupby(['iyear']).count().reset_index()
df3_ter = df2_ter.groupby('iyear')['eventid'].sum().reset_index().rename(columns = {'iyear' : 'year', 'eventid' : 'count'})
df4_ter = df2_ter.groupby('country_txt')['eventid'].sum().reset_index().rename(columns = {'country_txt' : 'country', 'eventid' : 'count'})
df4_ter
df3_ter

df5_ter = df2_ter[['iyear', 'country_txt', 'eventid', 'gname']].rename(columns = {'iyear' : 'year', 'country_txt' : 'country', 'eventid' : 'count', 'gname' : 'group'})
df5_ter

data_raw.drop(data_raw.loc[data_raw['Year']==1999].index, inplace=True)
data_raw.head(5) 

#Stap 2: Versimpel langere namen van landen om later toe te passen in de samenvoeging van datasets 
migranten_totaal_europa = data_raw[['Year', 'Country of asylum', "Refugees under UNHCR's mandate"]]
migranten_totaal_europa['Country of asylum'] = migranten_totaal_europa['Country of asylum'].replace({'Netherlands (Kingdom of the)': 'Netherlands'})
migranten_totaal_europa['Country of asylum'] = migranten_totaal_europa['Country of asylum'].replace({'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom'})
migranten_totaal_europa['Country of asylum'] = migranten_totaal_europa['Country of asylum'].replace({'Serbia and Kosovo: S/RES/1244 (1999)': 'Serbia'})
migranten_totaal_europa.head(5) 

data_ter['Country of asylum'] = data_ter['Country of asylum'].replace({'Macedonia': 'North Macedonia'})
data_ter['Country of asylum'] = data_ter['Country of asylum'].replace({'Slovak Republic': 'Slovakia'})
data_ter['Country of asylum'] = data_ter['Country of asylum'].replace({'Serbia-Montenegro': 'Serbia'})
data_ter = data_ter[['Country of asylum', 'Year', 'Terrorist attacks']]
data_ter.head(5)

data_lon_lat['Country of asylum'] = data_lon_lat['Country of asylum'].replace({'Macedonia [FYROM]': 'North Macedonia'})
data_lon_lat2 = data_lon_lat[['latitude', 'longitude', 'Country of asylum']]
data_lon_lat2.head(5)

#Stap 3: Maak een nieuwe dataframe 
df2 = migranten_totaal_europa.groupby(['Year','Country of asylum'], as_index=False)["Refugees under UNHCR's mandate"].sum()
print(df2)

df3 = migranten_totaal_europa.groupby(['Year'], as_index=False)["Refugees under UNHCR's mandate"].sum()
print(df3)

df4 = migranten_totaal_europa.groupby(['Country of asylum'], as_index=False)["Refugees under UNHCR's mandate"].sum()
print(df4)

df3_ter
sum(df3_ter['count'])

#Stap 4: voeg de data van terroristische aanslagen en aantal vluchtelingen samen 
df_half = df2.merge(data_lon_lat2, on='Country of asylum')
df_full = pd.merge(df_half, data_ter,  how='left', left_on=['Country of asylum','Year'], right_on = ['Country of asylum','Year']).fillna(0)
df_full.head(500) 
#df_full.to_csv('test.csv')

#Stap 5: maak map figuur 
fig = px.scatter_geo(df_full, 
                     lat='latitude',
                     lon='longitude',
                     color="Refugees under UNHCR's mandate",
                     color_continuous_scale=px.colors.sequential.OrRd,
                     height=650,
                     width=950,
                     hover_name="Country of asylum",
                     hover_data={
                                "Refugees under UNHCR's mandate":True, 
                                "Terrorist attacks":True,
                                "latitude":False, 
                                "longitude":False,
                                },
                     projection="natural earth",
                     scope="europe",
                     size=df_full["Year"],
                     size_max=10,
                     animation_frame="Year",
                     range_color=(0,200000),
                    )


fig.update_geos(
    visible=True, 
    resolution=50,
    showcountries=True, 
    countrycolor="whitesmoke",
    bgcolor='whitesmoke',
    landcolor='#26a783',
)

fig.update_layout(
    title='Hoeveelheid vluchtelingen en terroristische aanvallen tussen 2000 en 2017',
)

fig.update_traces(marker=dict(line=dict(width=0.5, color='black')))
fig.show()

#Stap 6: maak de nieuwe files voor de datastory 
# df2.to_csv('migranten_per_jaar_per_land.csv')
# df3.to_csv('migranten_per_jaar_europa.csv')
# df4.to_csv('migranten_per_land_totaal.csv')
# df3_ter.to_csv('aanslagen_per_jaar.csv', index=False, header=True)
# df4_ter.to_csv('totale_aanslagen_per_land.csv', index=False, header=True)
# df5_ter.to_csv('aanslagen_per_jaar_per_land_per_groep.csv', index=False, header=True)

FileNotFoundError: [Errno 2] No such file or directory: 'migranten_eu.csv'